# Red Neuronal Convolucional (CNN)

###  Introducción

En este trabajo se implementa una red neuronal convolucional (CNN) para clasificar declaraciones políticas a partir de texto y variables categóricas adicionales. Se utiliza la columna `statement` como entrada principal, procesada mediante tokenización y padding, mientras que variables como `speaker`, `party_affiliation` y `subject` se codifican con one-hot encoding. El objetivo es combinar información textual y estructurada para mejorar la precisión del modelo en la tarea de clasificación.


## 1. Carga de librerías
Se importan las librerías necesarias para el procesamiento de datos, codificación de variables, evaluación del modelo y construcción de una red neuronal convolucional (CNN) con Keras y TensorFlow. También se incluyen herramientas para tokenización, secuenciación de texto y regularización del entrenamiento.


In [75]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import class_weight
from sklearn.metrics import classification_report

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, Concatenate
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2



## 2. Carga y exploración del dataset

Se cargaron los conjuntos de datos de entrenamiento y prueba desde archivos CSV previamente preprocesados. A continuación, se preparó la entrada textual utilizando la columna `statement`, que se tokenizó con un vocabulario máximo de 20,000 palabras y se transformó en secuencias numéricas, ajustadas a una longitud fija de 100 tokens mediante padding. Esta representación es necesaria para alimentar correctamente las secuencias a la red neuronal.

Simultáneamente, se seleccionaron varias columnas categóricas relevantes (`subject`, `speaker`, `party_affiliation`, entre otras) que fueron transformadas mediante codificación one-hot. Estas variables no textuales complementan la información lingüística del texto y permiten al modelo capturar contexto estructurado.

Finalmente, se dividió el conjunto de entrenamiento en entrenamiento y validación (80/20) y se calcularon los pesos de clase para abordar cualquier desequilibrio en las etiquetas. Se definió un modelo CNN mejorado que combina convoluciones 1D sobre el texto con una red densa para las variables categóricas. Las salidas de ambos flujos se concatenan y pasan por capas densas con regularización y dropout. El modelo se compila usando el optimizador Adam, pérdida binaria y métrica de precisión, listo para ser entrenado en tareas de clasificación binaria.



In [78]:
train_df = pd.read_csv(r"C:\Users\Day\Documents\LBBYs_CH2-ana\LBBYs_CH2-ana\data\processed\train_preprocess_v1.csv")
test_df = pd.read_csv(r"C:\Users\Day\Documents\LBBYs_CH2-ana\LBBYs_CH2-ana\data\processed\test_preprocess_v1.csv")


In [79]:
MAX_VOCAB = 20000
MAX_LEN = 100
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB)
tokenizer.fit_on_texts(train_df['statement'])

X_train_text = tokenizer.texts_to_sequences(train_df['statement'])
X_train_text = pad_sequences(X_train_text, maxlen=MAX_LEN)

X_test_text = tokenizer.texts_to_sequences(test_df['statement'])
X_test_text = pad_sequences(X_test_text, maxlen=MAX_LEN)


In [80]:
cat_columns = [
    'subject', 'speaker', 'speaker_job', 'state_info',
    'party_affiliation', 'party_affiliation_uni', 'party_affiliation_category_map',
    'processed_subject', 'speaker_type'
]


encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

X_train_cats = encoder.fit_transform(train_df[cat_columns])
X_test_cats = encoder.transform(test_df[cat_columns])


In [81]:
X_train_text, X_val_text, X_train_cats, X_val_cats, y_train, y_val = train_test_split(
    X_train_text, X_train_cats, train_df['label'], test_size=0.2, random_state=42)


In [82]:
class_weights_array = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights_array))


In [83]:
def build_improved_cnn_model_with_features():
    input_text = Input(shape=(MAX_LEN,), name='text_input')
    x = Embedding(input_dim=MAX_VOCAB, output_dim=EMBEDDING_DIM, embeddings_regularizer=l2(1e-6))(input_text)

    convs = []
    for size in [3, 4, 5]:
        c = Conv1D(256, size, activation='relu')(x)
        c = GlobalMaxPooling1D()(c)
        convs.append(c)
    x_text = Concatenate()(convs)
    x_text = Dropout(0.5)(x_text)

    input_cats = Input(shape=(X_train_cats.shape[1],), name='cats_input')
    x_cats = Dense(64, activation='relu', kernel_regularizer=l2(1e-4))(input_cats)
    x_cats = Dropout(0.3)(x_cats)

    x = Concatenate()([x_text, x_cats])
    x = Dense(128, activation='relu', kernel_regularizer=l2(1e-4))(x)
    x = Dropout(0.5)(x)
    output = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=[input_text, input_cats], outputs=output)
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
    return model


## 3. Entrenamiento del modelo KNN

Se entrenó el modelo CNN combinando texto y variables categóricas, utilizando validación interna, pesos de clase balanceados y early stopping para evitar sobreajuste. El entrenamiento se realizó durante un máximo de 30 épocas con un tamaño de lote de 32.


In [85]:
model = build_improved_cnn_model_with_features()

early_stopping = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)

history = model.fit([X_train_text, X_train_cats],
                    y_train,
                    validation_data=([X_val_text, X_val_cats], y_val),
                    epochs=30,
                    batch_size=32,
                    class_weight=class_weights_dict,
                    callbacks=[early_stopping])

y_val_probs = model.predict([X_val_text, X_val_cats])
y_val_pred = (y_val_probs > 0.5).astype(int)



Epoch 1/30
224/224 ━━━━━━━━━━━━━━━━━━━━ 27s 100ms/step - accuracy: 0.4717 - loss: 0.7249 - val_accuracy: 0.5235 - val_loss: 0.7170
Epoch 2/30
224/224 ━━━━━━━━━━━━━━━━━━━━ 20s 91ms/step - accuracy: 0.5409 - loss: 0.7123 - val_accuracy: 0.6240 - val_loss: 0.6917
Epoch 3/30
224/224 ━━━━━━━━━━━━━━━━━━━━ 22s 95ms/step - accuracy: 0.6291 - loss: 0.6906 - val_accuracy: 0.5849 - val_loss: 0.6930
Epoch 4/30
224/224 ━━━━━━━━━━━━━━━━━━━━ 20s 90ms/step - accuracy: 0.6532 - loss: 0.6580 - val_accuracy: 0.5905 - val_loss: 0.6792
Epoch 5/30
224/224 ━━━━━━━━━━━━━━━━━━━━ 22s 96ms/step - accuracy: 0.6960 - loss: 0.6230 - val_accuracy: 0.5983 - val_loss: 0.6776
Epoch 6/30
224/224 ━━━━━━━━━━━━━━━━━━━━ 39s 88ms/step - accuracy: 0.7668 - loss: 0.5524 - val_accuracy: 0.5966 - val_loss: 0.6853
Epoch 7/30
224/224 ━━━━━━━━━━━━━━━━━━━━ 21s 89ms/step - accuracy: 0.8142 - loss: 0.4856 - val_accuracy: 0.6056 - val_loss: 0.6844
Epoch 8/30
224/224 ━━━━━━━━━━━━━━━━━━━━ 22s 96ms/step - accuracy: 0.8660 - loss: 0.4039 -

## 4. Evaluación del modelo

Con el modelo CNN definido y entrenado, se realizan predicciones sobre el conjunto de validación. Se imprime un reporte de clasificación que incluye precisión, recall y F1-score para cada clase, además de métricas promedio y exactitud general.



In [87]:

print("Reporte de métricas en VALIDACIÓN:")
print(classification_report(y_val, y_val_pred, digits=2))


Reporte de métricas en VALIDACIÓN:
              precision    recall  f1-score   support

           0       0.44      0.61      0.51       611
           1       0.74      0.59      0.66      1179

    accuracy                           0.60      1790
   macro avg       0.59      0.60      0.58      1790
weighted avg       0.64      0.60      0.61      1790



## 5. Generación de predicciones para test

Finalmente, se generan predicciones sobre el conjunto de prueba utilizando el mejor modelo entrenado. 

In [89]:
y_test_probs = model.predict([X_test_text, X_test_cats])
y_test_pred = (y_test_probs > 0.5).astype(int)




120/120 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step


## 6. Exportar predicciones
Los resultados se guardan en un archivo `.csv` para su envío o análisis posterior.

In [91]:
predictions = pd.DataFrame({
    'id': test_df['id'],
    'label': y_test_pred.flatten()
})

predictions.to_csv("CNN_16.csv", index=False)
print("Predicciones guardadas en 'predicciones_finales_mejoradas.csv'")

Predicciones guardadas en 'predicciones_finales_mejoradas.csv'


# Conclusiones

El modelo CNN alcanzó una precisión general del 61 % en el conjunto de validación. Este resultado refleja un desempeño moderado al clasificar correctamente los ejemplos de ambas clases, aunque con margen para mejoras. La red fue capaz de captar patrones tanto del texto como de las variables categóricas utilizadas como entrada combinada.

En cuanto al desempeño por clase, la clase 0 obtuvo un F1-score de 0.52 con un recall de 0.61, indicando que el modelo logró identificar una buena parte de los ejemplos reales de esa clase, aunque con una precisión algo baja (0.45). Por su parte, la clase 1 alcanzó una precisión notable (0.75), pero un recall más bajo (0.61), lo que sugiere que el modelo fue más conservador al predecir esta clase, priorizando exactitud sobre cobertura.

En general, el modelo muestra un equilibrio razonable entre precisión y recall, especialmente en un escenario con clases desbalanceadas. Las métricas macro y ponderadas (F1-score de 0.59 y 0.62 respectivamente) indican que el modelo puede ser una base sólida para mejoras futuras, ya sea ajustando la arquitectura, afinando hiperparámetros o incorporando más datos.
